In [2]:
import sys
import os

# Lấy đường dẫn thư mục project (data_mining_lab)
project_root = os.path.abspath("..")

# Thêm vào Python path
if project_root not in sys.path:
    sys.path.append(project_root)

print(project_root)

/Users/apple/Desktop/data_mining_lab


In [3]:
from src.apriori_library import FPGrowthMiner

In [5]:
import pandas as pd

basket_bool = pd.DataFrame({
    "Milk":   [1, 1, 0, 1, 0],
    "Bread":  [1, 1, 1, 0, 1],
    "Butter": [0, 1, 1, 1, 0],
    "Beer":   [0, 0, 1, 1, 1]
}).astype(bool)

basket_bool

,Milk,Bread,Butter,Beer
0,True,True,False,False
1,True,True,True,False
2,False,True,True,True
3,True,False,True,True
4,False,True,False,True


In [6]:
fp_miner = FPGrowthMiner()

freq_itemsets, rules, exec_time = fp_miner.run(
    basket_bool=basket_bool,
    min_support=0.4,
    min_confidence=0.6,
    min_lift=1.0
)

rules


FP-Growth finished in 0.07 seconds
Frequent itemsets: 9
Association rules: 4


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
1,(Butter),(Milk),0.6,0.6,0.4,0.666667,1.111111,1.0,0.04,1.2,0.25,0.5,0.166667,0.666667
2,(Milk),(Butter),0.6,0.6,0.4,0.666667,1.111111,1.0,0.04,1.2,0.25,0.5,0.166667,0.666667
4,(Beer),(Butter),0.6,0.6,0.4,0.666667,1.111111,1.0,0.04,1.2,0.25,0.5,0.166667,0.666667
5,(Butter),(Beer),0.6,0.6,0.4,0.666667,1.111111,1.0,0.04,1.2,0.25,0.5,0.166667,0.666667


In [7]:
import os
os.listdir("../data")

['online_retail.csv']

In [10]:
df = pd.read_csv(
    "../data/online_retail.csv",
    encoding="ISO-8859-1",
    sep=";"
)

df.head()


,ï»¿InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01/12/2010 08:26,"2,55",17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01/12/2010 08:26,"3,39",17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01/12/2010 08:26,"2,75",17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01/12/2010 08:26,"3,39",17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01/12/2010 08:26,"3,39",17850.0,United Kingdom


In [11]:
df.columns

Index(['ï»¿InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [15]:
# Ép kiểu số cho Quantity và UnitPrice
df["Quantity"] = pd.to_numeric(df["Quantity"], errors="coerce")
df["UnitPrice"] = pd.to_numeric(df["UnitPrice"], errors="coerce")

df.dtypes


ï»¿InvoiceNo     object
StockCode        object
Description      object
Quantity          int64
InvoiceDate      object
UnitPrice       float64
CustomerID      float64
Country          object
dtype: object

In [16]:
# Bỏ dòng thiếu dữ liệu
df = df.dropna()

# Chỉ giữ giao dịch hợp lệ
df = df[df["Quantity"] > 0]
df = df[df["UnitPrice"] > 0]

df.shape


(1712, 8)

In [18]:
# Xoá ký tự BOM và khoảng trắng thừa trong tên cột
df.columns = df.columns.str.replace("ï»¿", "", regex=False).str.strip()

df.columns


Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [19]:
basket = (
    df
    .groupby(["InvoiceNo", "Description"])["Quantity"]
    .sum()
    .unstack()
    .fillna(0)
)

basket.head()


Description,BEADED CHANDELIER T-LIGHT HOLDER,BILI NUT AND WOOD NECKLACE,BLING KEY RING STAND,BOTANICAL GARDENS WALL CLOCK,BREAD BIN DINER STYLE PINK,BREAD BIN DINER STYLE RED,BROWN CHECK CAT DOORSTOP,BROWN KUKUI COCONUT SEED NECKLACE,Bank Charges,CAKE STAND VICTORIAN FILIGREE MED,...,SMALL WHITE RETROSPOT MUG IN BOX,SPOTTED WHITE NATURAL SEED NECKLACE,TWO DOOR CURIO CABINET,UTILTY CABINET WITH HOOKS,VANILLA SCENT CANDLE JEWELLED BOX,VICTORIAN SEWING BOX SMALL,VINTAGE BLUE KITCHEN CABINET,VINTAGE RED KITCHEN CABINET,VINTAGE WOODEN BAR STOOL,WOODEN ADVENT CALENDAR CREAM
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536392,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536403,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536540,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# Tạo basket_bool
basket_bool = basket.applymap(lambda x: x > 0)

# Giảm số sản phẩm (tuỳ chọn nhưng nên làm)
item_support = basket_bool.sum()
frequent_items = item_support[item_support >= 50].index
basket_bool = basket_bool[frequent_items]

# Chạy FP-Growth
from src.apriori_library import FPGrowthMiner

fp_miner = FPGrowthMiner()
freq_itemsets_fp, rules_fp, time_fp = fp_miner.run(
    basket_bool=basket_bool,
    min_support=0.01,
    min_confidence=0.3,
    min_lift=1.2
)

rules_fp.head()


FP-Growth finished in 0.02 seconds
Frequent itemsets: 4
Association rules: 0


/var/folders/_f/jpf53z4n0rj8rmdnbygcl6wc0000gn/T/ipykernel_4998/1295799238.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_bool = basket.applymap(lambda x: x > 0)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski


In [21]:
freq_itemsets_fp, rules_fp, time_fp = fp_miner.run(
    basket_bool=basket_bool,
    min_support=0.005,      # giảm support
    min_confidence=0.2,     # giảm confidence
    min_lift=1.0            # lift >= 1
)

rules_fp.head(10)


FP-Growth finished in 0.02 seconds
Frequent itemsets: 4
Association rules: 0


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski


In [29]:
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules
import time


In [25]:
start_time = time.time()

freq_itemsets_ap = apriori(
    basket_bool,
    min_support=0.005,
    use_colnames=True
)

rules_ap = association_rules(
    freq_itemsets_ap,
    metric="confidence",
    min_threshold=0.2
)

rules_ap = rules_ap[rules_ap["lift"] >= 1.0]

time_ap = time.time() - start_time

print(f"Apriori finished in {time_ap:.2f} seconds")
print(f"Frequent itemsets (Apriori): {len(freq_itemsets_ap)}")
print(f"Association rules (Apriori): {len(rules_ap)}")


Apriori finished in 0.01 seconds
Frequent itemsets (Apriori): 4
Association rules (Apriori): 0


In [26]:
fp_miner = FPGrowthMiner()

freq_itemsets_fp, rules_fp, time_fp = fp_miner.run(
    basket_bool=basket_bool,
    min_support=0.005,
    min_confidence=0.2,
    min_lift=1.0
)


FP-Growth finished in 0.02 seconds
Frequent itemsets: 4
Association rules: 0


In [27]:
import pandas as pd

comparison = pd.DataFrame({
    "Algorithm": ["Apriori", "FP-Growth"],
    "Execution Time (s)": [time_ap, time_fp],
    "Frequent Itemsets": [len(freq_itemsets_ap), len(freq_itemsets_fp)],
    "Association Rules": [len(rules_ap), len(rules_fp)]
})

comparison


,Algorithm,Execution Time (s),Frequent Itemsets,Association Rules
0,Apriori,0.013754,4,0
1,FP-Growth,0.020438,4,0


In [30]:
supports = [0.02, 0.01, 0.005]

results = []

for sup in supports:
    # Apriori
    t0 = time.time()
    fi_ap = apriori(basket_bool, min_support=sup, use_colnames=True)
    t_ap = time.time() - t0

    # FP-Growth
    t0 = time.time()
    fi_fp = fpgrowth(basket_bool, min_support=sup, use_colnames=True)
    t_fp = time.time() - t0

    results.append({
        "min_support": sup,
        "Apriori_time": t_ap,
        "FP_time": t_fp,
        "Apriori_itemsets": len(fi_ap),
        "FP_itemsets": len(fi_fp)
    })

pd.DataFrame(results)


,min_support,Apriori_time,FP_time,Apriori_itemsets,FP_itemsets
0,0.020,0.007020,0.049627,4,4
1,0.010,0.018541,0.056247,4,4
2,0.005,0.002799,0.017799,4,4
